In [151]:
#@markdown ##import
#from google.colab import drive
%matplotlib notebook
#%matplotlib ipympl
#%matplotlib widget
#%matplotlib interactive
#%matplotlib inline
import trackpy
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
#import sdt
#from sdt import io, chromatic, multicolor, brightness
from mpl_toolkits.mplot3d import Axes3D
import scipy.io
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler

In [43]:
#@markdown ##mount drive

#drive.mount('/gdrive', force_remount=True)

In [152]:
#@markdown ## import data

## google drive
#data = pd.read_csv( '/gdrive/MyDrive/1 UNI/BIO/210422_red_npc/cell1_tr1000_def500.csv',
#                   header = None, names=["x", "y", "z", "n", "bg","fit","id","frame"])
#drift = pd.read_csv('/gdrive/MyDrive/1 UNI/BIO/210422_red_npc/day2_cell1_driftValues.csv')

## local
data = pd.read_csv( 'data/210422_npc/cell2_tr800_def500.csv',
                   header = None, names=["x", "y", "z", "n", "bg","fit","id","frame"])
drift = pd.read_csv('data/210422_npc/cell2_driftValues.csv')

fig = plt.figure()
ax = fig.add_subplot( projection = '3d' )
f = ax.scatter(data['x'], data['y'], data['z'], s=1 ,c=data['n'])
ax.set_xlabel('x / nm')
ax.set_ylabel('y / nm')
ax.set_zlabel('z / nm')
plt.title('All localizations')
cbar = plt.colorbar( f, label = 'Photons' )
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [84]:
#@markdown ## import & scale drift

drift['Y2']=drift['Y2']*146.6
drift['Y3']=drift['Y3']*146.6
drift['X2']=np.round(drift['X2'])
drift['X3']=np.round(drift['X3'])

drift

,X0,Y0,Y1,X2,Y2,X3,Y3
0,2205.5,0.000000,0.000000,1.0,10.248366,1.0,3.840920
1,7524.0,-0.135277,-0.060043,251.0,9.086121,251.0,3.405371
2,15006.0,-0.105056,-0.117081,501.0,7.923877,501.0,2.969823
3,25566.0,-0.022300,-0.143226,751.0,6.761778,751.0,2.534274
4,40879.0,-0.030004,-0.130091,1001.0,5.599534,1001.0,2.098579
...,...,...,...,...,...,...,...
195,NaN,NaN,NaN,48750.0,-7.473082,48750.0,-17.329000
196,NaN,NaN,NaN,49000.0,-7.570717,49000.0,-17.273731
197,NaN,NaN,NaN,49250.0,-7.668353,49250.0,-17.218317
198,NaN,NaN,NaN,49500.0,-7.765988,49500.0,-17.163048


In [101]:
#@markdown ## apply drift

for i in range(len(drift)-1):
    fr=data[(data['frame']>=drift['X2'].iloc[i]) & (data['frame']<drift['X2'].iloc[i+1])]
    fr['y']=fr['y']-drift['Y2'].iloc[i]
    fr['x']=fr['x']-drift['Y3'].iloc[i]
    data[(data['frame']>=drift['X2'].iloc[i]) & (data['frame']<drift['X2'].iloc[i+1])]=fr                                                             

<ipython-input-101-ef8f02d48c68>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fr['y']=fr['y']-drift['Y2'].iloc[i]
<ipython-input-101-ef8f02d48c68>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fr['x']=fr['x']-drift['Y3'].iloc[i]
<ipython-input-101-ef8f02d48c68>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gui

In [102]:
#@markdown ## plot drift corrected data

fig=plt.figure()
ax = fig.add_subplot( projection = '3d' )
f=ax.scatter(data['x'], data['y'], data['z'], s=1 ,c=data['n'])
ax.set_xlabel('x / nm')
ax.set_ylabel('y / nm')
ax.set_zlabel('z / nm')
plt.title('All localizations')
cbar=plt.colorbar(f, label='Photons')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [103]:
#@markdown ## histogram
#@markdown **range=(0,20000)**

fig = plt.figure()
plt.hist( data['n'], bins=50, range=(0,20000) )
plt.title('N photon number')
plt.xlabel('N / photons')
plt.ylabel('counts')
plt.tight_layout


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<function matplotlib.pyplot.tight_layout(*, pad=1.08, h_pad=None, w_pad=None, rect=None)>

In [88]:
#@markdown ## filter
#@markdown - **2k < photons < 7k**
#@markdown - **0 < z < 499**
#@markdown - **fit < 6M**

fdata = data[ (data['n']>2000) & (data['n']<7000) & (data['z']>0) & (data['z']<499) & (data['fit']<6e6) ]

fig = plt.figure()
ax = fig.add_subplot( projection = '3d' )
ff = ax.scatter( fdata['x'], fdata['y'], fdata['z'], s=1 ,c=fdata['n'] )
ax.set_xlabel('x / nm')
ax.set_ylabel('y / nm')
ax.set_zlabel('z / nm')
plt.title('filtered localizations' )
cbar = plt.colorbar( ff, label = 'Photons' )

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [136]:
#@markdown ## track all in x,y

sr = 50 #100
mem = 10 #100

#linkedxy = trackpy.link_df( fdata, pos_columns = ["x","y","z"], search_range = sr,  memory = mem )

particles = linkedxy.groupby( "particle" ).aggregate( np.mean )
std_pos = linkedxy.groupby( "particle" ).aggregate( 'std' )
particles["length"] = linkedxy.groupby( "particle" ).apply( len )
particles["z_std"] = std_pos['z'].copy()
particles["x_std"] = std_pos['x'].copy()
particles["y_std"] = std_pos['y'].copy()

particles

,x,y,z,n,bg,fit,id,frame,length,z_std,x_std,y_std
particle,,,,,,,,,,,,
0,12391.739048,6815.250724,305.346448,2462.328748,53.316888,2.876004e+06,31.0,1.0,1,NaN,NaN,NaN
1,7848.212724,11893.633453,305.733528,4984.304107,74.304594,4.610566e+06,68.0,1.0,1,NaN,NaN,NaN
2,12365.655877,6807.420963,398.709373,2811.310034,55.941528,3.106258e+06,154.0,2.0,1,NaN,NaN,NaN
3,10902.644257,17720.013607,234.955544,2465.103085,71.124857,3.789877e+06,224.0,2.0,1,NaN,NaN,NaN
4,3965.565509,11354.521724,167.031299,5215.103109,65.771847,4.181920e+06,297.0,3.0,1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
60339,11366.910064,5773.755680,338.948625,2201.858977,29.651125,1.655082e+06,505771.0,49984.0,1,NaN,NaN,NaN
60340,10498.949071,16700.155695,260.420482,3978.455882,33.044771,2.233162e+06,505772.0,49984.0,1,NaN,NaN,NaN
60341,18437.493205,14427.018376,364.186896,2304.117453,30.914944,1.742087e+06,505783.0,49991.0,1,NaN,NaN,NaN


In [145]:
#@markdown ## cut down to tracked particles

localisations = particles

## select first 10000 rows & 4 columns using iloc selector
#localisations = fdata.iloc[0:10000,0:4]

fig = plt.figure()
ax = fig.add_subplot( projection = '3d' )
ff = ax.scatter( localisations['x'], localisations['y'], localisations['z'], s=1 ,c=localisations['n'] )
ax.set_xlabel('x / nm')
ax.set_ylabel('y / nm')
ax.set_zlabel('z / nm')
plt.title('filtered localizations')
cbar = plt.colorbar( ff, label = 'photons' )
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [140]:
#@markdown ## compute dbscan

alocalisations = localisations.to_numpy()
alocalisations[ :, 0:2 ]

##db = DBSCAN( eps = 300, min_samples=10 ).fit( alocalisations[:,0:3] ) # cluster 3d
db = DBSCAN( eps = 200, min_samples = 10 ).fit( alocalisations[ :, 0:2 ] ) # cluster 2d
core_samples_mask = np.zeros_like( db.labels_, dtype=bool )
core_samples_mask[ db.core_sample_indices_ ] = True
labels = db.labels_
localisations[ "cluster" ] = labels

# count clusters (ignore noise if present)
n_clusters_ = len( set( labels ) ) - ( 1 if -1 in labels else 0 )
n_noise_ = list( labels ).count( -1 )

#print('Estimated number of clusters: %d' % n_clusters_)
#print('Estimated number of noise points: %d' % n_noise_)

nlocalisations = localisations.loc[ localisations['cluster'] == -1 ]
clocalisations = localisations.loc[ localisations['cluster'] != -1 ]

fig = plt.figure()
ax = fig.add_subplot( projection = '3d' )
ff = ax.scatter( clocalisations['x'], clocalisations['y'], clocalisations['z'], s=1 ,c=clocalisations['n'] )
ff = ax.scatter( nlocalisations['x'], nlocalisations['y'], nlocalisations['z'], s=1 ,c="black" )
ax.set_xlabel('x / nm')
ax.set_ylabel('y / nm')
ax.set_zlabel('z / nm')
plt.title('{0} {1} {2} {3}'.format( 'Estimated number of clusters/noise: ', n_clusters_, '/', n_noise_ ) )
cbar = plt.colorbar( ff, label = 'Photons' )
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [146]:
#@markdown ## analyse clusters

clabels = set(labels)
cnames = [ "counts", "xmean", "ymean", "zmean", "nmean", "xvar", "yvar", "zvar", "nvar", "label" ]
clusters = pd.DataFrame( index = clabels, columns = cnames, dtype="float64" )
clusters[ "label" ] = clabels

for k in clabels:
  tmp = localisations.loc[ localisations['cluster'] == k ]
  clusters.loc[ k, "counts" ] = len( tmp )
  for label in [ "x", "y", "z", "n" ]:
    clusters.loc[ k, label+"mean"  ] = np.mean( tmp.loc[ :, label ] )
    clusters.loc[ k, label+"var"  ] = np.var( tmp.loc[ :, label ] )

print( * [ 'analysed clusters: ', len( clusters ) ] )


analysed clusters:  76


In [147]:
#@markdown ## filter clusters based on variance
#@markdown - **x,y variance < 10k**
#@markdown - **z variance < 10k**

## set thresholds
xythresh = 1e4
zthresh = 1e4

fclusters = clusters[ ( clusters['xvar'] < xythresh ) & 
                     ( clusters['yvar'] < xythresh ) &
                     ( clusters['zvar'] < zthresh) ]

fig = plt.figure()
ax = fig.add_subplot( projection = '3d' )
ff = ax.scatter( fclusters['xmean'], fclusters['ymean'], fclusters['zmean'], s=1 ,c=fclusters['counts'] )
ax.set_xlabel('x / nm')
ax.set_ylabel('y / nm')
ax.set_zlabel('z / nm')
plt.title( 'filtered clusters: ' + str( len( fclusters ) ) )
cbar = plt.colorbar( ff, label='counts' ) 

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [148]:
#@markdown ## select 10 best clusters & plot localisations

## sort by variance
#sclusters = fclusters.sort_values( "xvar" )

## sort by x and y variance using least squares
sclusters = fclusters.loc[ ( fclusters.xvar ** 2 + fclusters.yvar ** 2 ).sort_values().index ]
selected_clusters = sclusters["label"].iloc[0:99]

fig = plt.figure()
ax = fig.add_subplot( projection = '3d' )
for clus in selected_clusters:
  flocalisations = localisations[ ( localisations['cluster'] == clus ) ]
  ff = ax.scatter( flocalisations['x'], flocalisations['y'], flocalisations['z'], s=1 ,c = flocalisations['n'] )

ax.set_xlabel('x / nm')
ax.set_ylabel('y / nm')
ax.set_zlabel('z / nm')
plt.title( "localizations of selected clusters" )
cbar = plt.colorbar( ff, label = 'photons' )

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [149]:
#@markdown ## plot localisations as clusters

sclusters['color'] = ""
vec = np.linspace( 0, 1, len( selected_clusters ) )
for each in range( len( selected_clusters ) ):
  sclusters['color'].iloc[ each ] = plt.cm.Spectral( vec[ each ] )

fig = plt.figure()
ax = fig.add_subplot( projection = '3d' )
for clus in selected_clusters:
  flocalisations = localisations[ ( localisations['cluster'] == clus ) ]
  #print( tuple(  sclusters.loc[clus, "color" ] ) )
  ff = ax.scatter( flocalisations['x'], flocalisations['y'], flocalisations['z'],
                  s=1 ,color = tuple(  sclusters.loc[ clus, "color" ] ) )

ax.set_xlabel('x / nm')
ax.set_ylabel('y / nm')
ax.set_zlabel('z / nm')
plt.title( "localizations of selected clusters" )
cbar = plt.colorbar( ff, label = 'cluster' )

/usr/lib/python3.9/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [75]:
#@markdown ## optional: filter one cluster based on mean location and size

cluster = 3 # select cluster
size = 300 # select cluster size

flocalisations = localisations[ ( localisations['x'] > fclusters.loc[ :, "xmean" ].iloc[ cluster ] - size ) &
                               ( localisations['x'] < fclusters.loc[ :, "xmean" ].iloc[ cluster ] + size ) &
                               ( localisations['y'] > fclusters.loc[ :, "ymean" ].iloc[ cluster ] - size ) &
                               ( localisations['y'] < fclusters.loc[ :, "ymean" ].iloc[ cluster ] + size )
                               #( localisations['z'] > fclusters.loc[ :, "xmean" ].iloc[ cluster ] - size ) &
                               #( localisations['z'] < fclusters.loc[ :, "xmean" ].iloc[ cluster ] + size ) &
]

flocalisations


,x,y,z,n,cluster
325,7569.132247,3688.547536,313.502028,2049.989597,19
343,7579.786586,3693.417084,290.184008,2085.650079,19
371,7586.345064,3777.508890,204.903188,2793.272179,19
382,7586.208458,3784.610007,206.197579,3467.029225,19
403,7584.491061,3929.364836,148.584437,4404.794820,19
425,7588.428803,3629.603880,144.438679,4854.129185,19
457,7594.885289,3653.343100,129.886473,4617.313374,19
810,7572.197854,3703.730605,306.517480,2056.248797,19
1112,7587.364725,3700.917656,298.661127,2317.850750,19
1367,7611.878323,3628.314366,284.590843,2031.818367,19
